In [32]:
import os, sys, argparse, paramiko, yaml
from getpass import getpass
from os.path import exists

In [33]:
# Checks to see if the file specified for the results exists. Prompts the user that the file exists and if they want to delete it. If so, then the user provides the response,delete the file.

if exists('results.txt') is True:
    check = input('Results.txt already exists. Would you like to delete it? (y/n): ')
    if check == 'y':
        os.remove('results.txt')
    elif check == 'n':
        print('You chose to not delete the existing file, new resutls will append to the old results file.')
    else:
        print('Invalid Option')
        sys.exit()

In [34]:
# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password:  ")

# Host information
host = "127.0.0.1"
port = 22
username = "sarahfornaldes"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication failed")

In [39]:
# Start SFTP Session
sftp = ssh.open_sftp()

# Upload the fs.py and searchTerms yaml file
uploadFiles = ['/Users/sarahfornaldes/Desktop/SYS-320-01/Week6/Task3_fs.py', '/Users/sarahfornaldes/Desktop/SYS-320-01/Week6/Task2_searchTerms.yaml']

for eachFile in uploadFiles:
    sftp.put(eachFile, '/Users/sarahfornaldes/Desktop/SYS-320-01/Week6')

# Run the script in /usr/bin
command = ['python3 /Users/sarahfornaldes/Desktop/SYS-320-01/Week6/Task3_fs.py -d /usr/bin']

for eachCMD in command:
    # Get the output from the command
    stdin, stdout, stderr  = ssh.exec_command(eachCMD)
    print(stdout)
    # Get results  from stdout
    lines =  stdout.readlines()
    #print(lines)

    # Convert the list to a string
    output =  ''.join(lines)
    print(output)

    # Header output
    sepHeader = '' + '### BEGIN ' + eachCMD + ' ###\n\n'

    # Footer output
    sepFooter = '' + '### END ' + eachCMD + ' ###\n\n'

    # Concatonate the header, output, and footer
    cmd_output =  sepHeader + output +  sepFooter
    #print(cmd_output)

    # Save the cmd_output to a file
    with open('results.txt', 'a') as f:
        f.write(cmd_output)

# End SFTP Session
sftp.close()


#python3 /Users/sarahfornaldes/Desktop/SYS-320-01/Week6/Homework/Task3_fs.py -d /usr/bin

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sarahfornaldes/Desktop/SYS-320-01/Week6/Task3_fs.py'